In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Senior Design/"

Mounted at /content/drive


In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 14.3 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 880 kB 64.5 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 85.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c1dcf7b4ecf73dec064984657892e4e54cfb4ab9fa2190d7651b7af3df8a02c5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import csv
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import random
from sklearn.model_selection import train_test_split

model_path = path + "models/roberta-large-finetuned-qqp"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at /content/drive/My Drive/Senior Design/models/roberta-large-finetuned-qqp were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
data_path = path + "data/datascience_qa.json"

qa_data = {}

with open(data_path, "r", encoding='utf-8') as f:
    qa_data = json.load(f)

questions = []

for i in qa_data:
  question = qa_data[i]["question:"]
  answer = qa_data[i]["answer:"]
  
  query= str(question).strip()
  answer = str(answer).strip()

  if answer != "nan":
    # len(tokenizer(query)['input_ids'])
    questions.append(query)

In [6]:
# Split dataset into training and validation set.
train_ques, valid_ques = train_test_split(
    questions, train_size=0.99, shuffle=True, random_state=47)

In [7]:
len(train_ques)

13597

In [6]:
all_preds = []
no_overlap = []
overlap = 0
for i, ques in tqdm(enumerate(valid_ques)):
    print(f"Question: {i+1}/{len(valid_ques)}")
    for i in range(0, len(train_ques), 3):
      if i + 2 >= len(train_ques):
        break
      batch = [(ques, x) for x in train_ques[i:i+3]]
      inputs = tokenizer.batch_encode_plus(
          batch,
          add_special_tokens=True,
          truncation=True,
          padding=True,
          return_tensors="pt",
      )
      inputs["input_ids"] = inputs["input_ids"].cuda()
      inputs["attention_mask"] = inputs["attention_mask"].cuda()
      logits = model(**inputs)[0]
      preds = torch.argmax(logits, dim=1).tolist()
      probs = torch.softmax(logits, dim=1).tolist()
      all_preds.append(preds)
      if max(preds) > 0.5:
        print(batch)
        overlap += 1

    print("Matching questions = {:d}".format(overlap))
print("Matching questions = {:d}".format(overlap))

Output hidden; open in https://colab.research.google.com to view.